Name: Bailey Griswold
NYU ID: bg1672
Class: Data Science for Quantitative Finance
Date: September 12, 2017

Assignment 1

**Part 1: Plotting data with FIPS codes**


A) Download the State-County FIPS code data from the census website:
● https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt
● data specification is here: https://www.census.gov/geo/reference/codes/cou.html


B) And push it into a data object with, STATE, STATEFP, COUNTYFP, COUNTYNAME,
CLASSFP columns. Add a ‘FIPS’ column were FIPS=int(str(STATEFP)+str(COUNTYFP)).


C) Finally, add a ‘DUMMY’ column equal to 1 when STATE=’NY’ else 0.


D) Next, instantiate a US county map in bqplot by following a version of this example:
● https://github.com/bloomberg/bqplot/blob/master/bqplot-election.gif


E) Analogously to the above example, set the ‘color’ attribute of the map to be
color=dict(zi(data[‘FIPS’], data[‘DUMMY’])). Display the resulting map.

In [192]:
import pandas as pd
import numpy as np
from bqplot import pyplot as plt
from bqplot import *

In [212]:
!curl -O "https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt"
fips = pd.read_csv('national_county.txt', header = None, dtype={1: str, 2:str} )
fips.head()

,0,1,2,3,4
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [213]:
fips.rename(columns={0:'STATE', 1:'STATEFP',\
                     2: 'COUNTYFP', 3:'COUNTYNAME', 4:'CLASSFP'}, inplace = True)
fips.head()

,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [214]:
fips['FIPS'] = (fips['STATEFP'].astype(str) + fips['COUNTYFP'].astype(str)).astype(int)

In [215]:
fips.head()

,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP,FIPS
0,AL,01,001,Autauga County,H1,1001
1,AL,01,003,Baldwin County,H1,1003
2,AL,01,005,Barbour County,H1,1005
3,AL,01,007,Bibb County,H1,1007
4,AL,01,009,Blount County,H1,1009


In [216]:
fips['DUMMY'] = 0

In [217]:
fips.head()

,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP,FIPS,DUMMY
0,AL,01,001,Autauga County,H1,1001,0
1,AL,01,003,Baldwin County,H1,1003,0
2,AL,01,005,Barbour County,H1,1005,0
3,AL,01,007,Bibb County,H1,1007,0
4,AL,01,009,Blount County,H1,1009,0


In [218]:
fips['DUMMY'][fips.STATE == 'NY'] = 1

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [219]:
map_fig = plt.figure(title='Visualizing the 2016 Election Results', min_width=1300, min_height=800)
map_tt = Tooltip(fields=['name', 'color'], labels=['County Name'])
map_res = plt.geo(map_data=topo_load('map_data/USCountiesMap.json'), stroke_color='black',
                 colors={'default_color': 'Black'},
                  scales={'projection': AlbersUSA(),
                         'color': ColorScale(colors=['Red', 'Gray', 'DeepSkyBlue'])},
                  color=dict(zip(fips['FIPS'], fips['DUMMY'])), tooltip=map_tt)
map_fig

A Jupyter Widget

### Part 2: Analyzing the HMDA dataset at county level

1) Download the 2012 HMDA dataset. Load it into a pandas dataframe. Compute the following
statistics:  
a) How many loans in each ‘action_taken_name’ category are there?  
b) If you group by ‘agency_abbr’, how many loans in each ‘action_taken_name’ category
are there? 

2) Construct a dataframe indexed by ‘agency_abbr’ and ‘action_taken_name’ and save it into a ‘agency_action_stats.pkl’ file  

3) Construct a dataframe named df_statecnty_stats indexed by the ‘msamd’ (which is the same as the FIPS state-county code) and containing the following columns:  
a) median_hud_inc = county median of ‘hud_median_family_income’  
b) median_inc = median county income of loan applicants  
c) mean_inc = mean county income of loan applicants  
d) std_inc = std county income of loan applicants  
e) mean_loan_to_inc = county mean loan value to annual income  
f) ‘mean_income_over_hud’ = county mean of applicant income / hud_median_family_income  
g) ‘pct_denied_apps’ = percentage of denied loan applications  
h) ‘pct_rentals’ = can you compute the percent rental units in each county?  
i) ‘pct_minority’ = percent minority population  
j) ‘ln_pop’ = log of the county population  

4) Plot one of these columns on a US county map as in Part 1.

Extra credit question: Can you have the user select which column to plot on the county map via
a Dropdown ipywidget?

In [4]:
#!curl -O "http://files.consumerfinance.gov/hmda/hmda_lar-2012.csv.gz"
hdma_12 = pd.read_csv('hmda_lar-2012.csv.gz')

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,16,17,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
hdma_12.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,173.0,5,NaN,...,IL,Illinois,77300.0,264,2153.0,1971.0,45.820000,7894.0,NaN,170.679993
1,1,Loan originated,5,NCUA,National Credit Union Administration,2,Not Hispanic or Latino,83.0,5,NaN,...,MI,Michigan,52100.0,116,1662.0,1271.0,3.340000,4315.0,NaN,102.760002
2,6,Loan purchased by the institution,9,CFPB,Consumer Financial Protection Bureau,4,Not applicable,70.0,7,NaN,...,AR,Arkansas,58200.0,159,1194.0,708.0,21.870001,4239.0,NaN,127.639999
3,6,Loan purchased by the institution,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,108.0,5,NaN,...,MN,Minnesota,83900.0,100,1927.0,1871.0,13.680000,4832.0,NaN,137.669998
4,1,Loan originated,3,FDIC,Federal Deposit Insurance Corporation,2,Not Hispanic or Latino,NaN,5,NaN,...,IN,Indiana,63800.0,267,1309.0,1160.0,4.680000,3612.0,NaN,139.100006


In [8]:
len(hdma_12)

18691551

# NEED TO GO BACK AND RERUN EVERTHING REPLACING HDMA_SMALL WITH HDMA_12 !!!!

In [295]:
hdma_small = 

In [13]:
hdma_small.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,173.0,5,NaN,...,IL,Illinois,77300.0,264,2153.0,1971.0,45.820000,7894.0,NaN,170.679993
1000,3,Application denied by financial institution,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,160.0,5,NaN,...,VT,Vermont,76700.0,100,1660.0,1408.0,3.530000,4081.0,NaN,113.629997
2000,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,NaN,5,NaN,...,FL,Florida,50500.0,60,8903.0,6713.0,7.290000,14286.0,2.79,112.050003
3000,3,Application denied by financial institution,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,NaN,5,NaN,...,TX,Texas,56700.0,204,794.0,357.0,71.059998,2215.0,NaN,83.070000
4000,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,101.0,5,NaN,...,NC,North Carolina,53300.0,38,2127.0,1785.0,11.650000,5467.0,NaN,150.169998


In [14]:
len(hdma_small)

18692

In [22]:
#a) How many loans in each ‘action_taken_name’ category are there?


hdma_small.action_taken_name.value_counts()

Loan originated                                        9817
Loan purchased by the institution                      3148
Application denied by financial institution            2733
Application withdrawn by applicant                     1517
Application approved but not accepted                   696
File closed for incompleteness                          560
Preapproval request denied by financial institution     167
Preapproval request approved but not accepted            54
Name: action_taken_name, dtype: int64

In [34]:
#b) If you group by ‘agency_abbr’, how many loans in each ‘action_taken_name’ category are there?
counts = hdma_small.groupby('agency_abbr').action_taken_name.value_counts()
counts

agency_abbr  action_taken_name                                  
CFPB         Loan originated                                        4118
             Loan purchased by the institution                      2640
             Application denied by financial institution            1552
             Application withdrawn by applicant                      477
             Application approved but not accepted                   287
             File closed for incompleteness                          242
             Preapproval request denied by financial institution      94
             Preapproval request approved but not accepted            36
FDIC         Loan originated                                        1031
             Application denied by financial institution             161
             Application withdrawn by applicant                      100
             Loan purchased by the institution                        58
             Application approved but not accepted         

In [35]:
#Construct a dataframe indexed by ‘agency_abbr’ and ‘action_taken_name’ and save it into a ‘agency_action_stats.pkl’ file
pd.DataFrame(counts).to_pickle('agency_action_stats.pkl')

3) Construct a dataframe named df_statecnty_stats indexed by the ‘msamd’ (which is the same as the FIPS state-county code) and containing the following columns:  
a) median_hud_inc = county median of ‘hud_median_family_income’  
b) median_inc = median county income of loan applicants  
c) mean_inc = mean county income of loan applicants  
d) std_inc = std county income of loan applicants  
e) mean_loan_to_inc = county mean loan value to annual income  
f) ‘mean_income_over_hud’ = county mean of applicant income / hud_median_family_income  
g) ‘pct_denied_apps’ = percentage of denied loan applications  
h) ‘pct_rentals’ = can you compute the percent rental units in each county?  
i) ‘pct_minority’ = percent minority population  
j) ‘ln_pop’ = log of the county population  

In [37]:
# trash later
hdma_small.columns

Index(['action_taken', 'action_taken_name', 'agency_code', 'agency_abbr',
       'agency_name', 'applicant_ethnicity', 'applicant_ethnicity_name',
       'applicant_income_000s', 'applicant_race_1', 'applicant_race_2',
       'applicant_race_3', 'applicant_race_4', 'applicant_race_5',
       'applicant_race_name_1', 'applicant_race_name_2',
       'applicant_race_name_3', 'applicant_race_name_4',
       'applicant_race_name_5', 'applicant_sex', 'applicant_sex_name',
       'application_date_indicator', 'as_of_year', 'census_tract_number',
       'co_applicant_ethnicity', 'co_applicant_ethnicity_name',
       'co_applicant_race_1', 'co_applicant_race_2', 'co_applicant_race_3',
       'co_applicant_race_4', 'co_applicant_race_5',
       'co_applicant_race_name_1', 'co_applicant_race_name_2',
       'co_applicant_race_name_3', 'co_applicant_race_name_4',
       'co_applicant_race_name_5', 'co_applicant_sex', 'co_applicant_sex_name',
       'county_code', 'county_name', 'denial_reason_1', 

In [299]:
hdma_small['county_code'] = hdma_small['county_code'].fillna(0.0).astype(int)

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [298]:
hdma_small.state_code

0           17.0
1000        50.0
2000        12.0
3000        48.0
4000        37.0
5000        29.0
6000        48.0
7000        37.0
8000         8.0
9000        18.0
10000       50.0
11000       13.0
12000       22.0
13000       36.0
14000       50.0
15000       32.0
16000       18.0
17000       48.0
18000       37.0
19000       48.0
20000       53.0
21000       24.0
22000       13.0
23000       42.0
24000        6.0
25000       12.0
26000       49.0
27000        9.0
28000        6.0
29000       37.0
            ... 
18662000    39.0
18663000    39.0
18664000    27.0
18665000    41.0
18666000    47.0
18667000    25.0
18668000     6.0
18669000     8.0
18670000    26.0
18671000    42.0
18672000    36.0
18673000    13.0
18674000    41.0
18675000    37.0
18676000    48.0
18677000    42.0
18678000    25.0
18679000    29.0
18680000    36.0
18681000     9.0
18682000    48.0
18683000     6.0
18684000     6.0
18685000    54.0
18686000    27.0
18687000    17.0
18688000    12.0
18689000    18

In [300]:
hdma_small['state_code'] = hdma_small['county_code'].fillna(0.0).astype(int)

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [306]:
hdma_small.loc[6000]

action_taken                                                                      1
action_taken_name                                                   Loan originated
agency_code                                                                       7
agency_abbr                                                                     HUD
agency_name                             Department of Housing and Urban Development
applicant_ethnicity                                                               2
applicant_ethnicity_name                                     Not Hispanic or Latino
applicant_income_000s                                                            98
applicant_race_1                                                                  2
applicant_race_2                                                                NaN
applicant_race_3                                                                NaN
applicant_race_4                                                            

In [305]:
hdma_small['state_code'].apply('{:0>2}'.format)

0           197
1000         07
2000         83
3000        469
4000         35
5000        183
6000        491
7000         23
8000         37
9000         03
10000        07
11000        57
12000        19
13000       103
14000        07
15000        03
16000        03
17000       439
18000        23
19000       439
20000        33
21000        37
22000        67
23000       121
24000        41
25000        83
26000        49
27000        13
28000        65
29000        03
           ... 
18662000    113
18663000     49
18664000     37
18665000     67
18666000    157
18667000     05
18668000     87
18669000     41
18670000     49
18671000     77
18672000     59
18673000     67
18674000     35
18675000    179
18676000     85
18677000     29
18678000     09
18679000    127
18680000     55
18681000     03
18682000     29
18683000     65
18684000     25
18685000     67
18686000     03
18687000     43
18688000     69
18689000     73
18690000    167
18691000    199
Name: state_code, dtype:

In [296]:
#hdma_small['fips'] = 
(hdma_small['state_code'].astype(str) + hdma_small['county_code'].astype(str)).astype(int)
#hdma_small.state_code
#hdma_small[['state_code', 'state_name']]

ValueError: invalid literal for int() with base 10: '17.0197.0'

In [247]:
#median_hud_inc = county median of ‘hud_median_family_income’
median_hud_inc = hdma_small.groupby('msamd').median().hud_median_family_income

In [264]:
hdma_small.groupby('msamd').median().applicant_income_000s

msamd
10180.0     88.5
10380.0    119.0
10420.0     72.0
10500.0     81.5
10580.0     83.0
10740.0     64.0
10780.0    118.5
10900.0     73.0
11020.0     95.0
11100.0     70.0
11180.0     70.0
11260.0     96.0
11300.0    166.5
11340.0     46.0
11460.0    104.0
11500.0     49.0
11540.0     72.5
11700.0     66.0
12020.0    127.0
12060.0     82.5
12100.0    120.0
12220.0     63.0
12260.0     67.0
12420.0     90.0
12540.0     76.5
12580.0     97.0
12620.0     55.5
12700.0    119.5
12940.0     69.0
12980.0     66.0
           ...  
46540.0     35.5
46660.0     24.0
46700.0     74.0
47020.0      NaN
47220.0     48.0
47260.0     69.5
47300.0     53.0
47380.0     56.0
47580.0     46.0
47644.0     79.0
47894.0    116.0
47940.0     80.5
48140.0     48.0
48300.0     84.0
48424.0     77.5
48540.0     62.0
48620.0     60.0
48660.0     83.5
48700.0     63.0
48864.0     64.5
48900.0     82.0
49020.0     56.0
49180.0     55.0
49340.0     89.5
49420.0     34.0
49500.0     26.0
49620.0     66.0
49660.0 

In [265]:
#b) median_inc = median county income of loan applicants

median_inc = hdma_small.groupby('msamd').median().applicant_income_000s

#c) mean_inc = mean county income of loan applicants

mean_inc = hdma_small.groupby('msamd').mean().applicant_income_000s

#d) std_inc = std county income of loan applicants

std_inc = hdma_small.groupby('msamd').std().applicant_income_000s

In [249]:
#e) mean_loan_to_inc = county mean loan value to annual income

hdma_small['loan_to_inc'] = hdma_small.loan_amount_000s / hdma_small.applicant_income_000s

mean_loan_to_inc = hdma_small.groupby('msamd').mean().loan_to_inc


/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [266]:
#‘mean_income_over_hud’ = county mean of applicant income / hud_median_family_income

hdma_small['income_over_hud'] = hdma_small.applicant_income_000s / hdma_small.hud_median_family_income

mean_income_over_hud = hdma_small.groupby('msamd').mean().income_over_hud

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [267]:
#g) ‘pct_denied_apps’ = percentage of denied loan applications

denied_loans = hdma_small.groupby('action_taken_name').size()["Application denied by financial institution"]

pct_denied_apps = denied_loans/len(hdma_small)*100

In [268]:
# ‘pct_rentals’ = can you compute the percent rental units in each county?
grouped = hdma_small.groupby(['msamd', 'census_tract_number'], as_index=False)
tracts = grouped.aggregate(np.mean)
tracts_summed = tracts.groupby('msamd').sum()

units = tracts_summed.number_of_1_to_4_family_units
units_owned = tracts_summed.number_of_owner_occupied_units

pct_rentals = ((units - units_owned) / units) * 100

In [269]:
#i) ‘pct_minority’ = percent minority population
hdma_small['minority_pop_size'] = (hdma_small.minority_population/100) * hdma_small.population

grouped = hdma_small.groupby(['msamd', 'census_tract_number'], as_index=False)
tracts = grouped.aggregate(np.mean)
tracts_summed = tracts.groupby('msamd').sum()

pct_minority = (tracts_summed.minority_pop_size / tracts_summed.population)*100

/Users/baileygriswold/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [270]:
df_statecnty_stats = pd.DataFrame(median_hud_inc)

In [271]:
df_statecnty_stats['median_inc'] = median_inc

In [272]:
df_statecnty_stats['mean_inc'] = mean_inc

In [273]:
df_statecnty_stats['std_inc'] = std_inc
df_statecnty_stats['mean_loan_to_inc'] = mean_loan_to_inc
df_statecnty_stats['mean_income_over_hud'] = mean_income_over_hud
df_statecnty_stats['pct_denied_apps'] = pct_denied_apps
df_statecnty_stats['pct_rentals'] = pct_rentals
df_statecnty_stats['pct_minority'] = pct_minority

df_statecnty_stats
 

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud,pct_denied_apps,pct_rentals,pct_minority
msamd,,,,,,,,,
10180.0,52900.0,88.5,88.500000,20.506097,1.158423,0.001673,14.621228,28.400994,19.662451
10380.0,16600.0,119.0,110.200000,80.862229,1.157336,0.006639,14.621228,40.691458,98.921266
10420.0,66500.0,72.0,82.838710,48.747716,1.897519,0.001246,14.621228,19.649796,11.968323
10500.0,49400.0,81.5,82.250000,29.881711,2.063765,0.001665,14.621228,26.099862,22.874824
10580.0,78100.0,83.0,95.068182,63.521246,2.085460,0.001217,14.621228,24.502761,12.073518
10740.0,61900.0,64.0,82.882353,57.968476,3.394150,0.001339,14.621228,25.018652,52.440586
10780.0,52400.0,118.5,270.333333,421.120252,1.221641,0.005159,14.621228,31.147279,37.154751
10900.0,73300.0,73.0,80.106383,44.816648,2.576662,0.001093,14.621228,21.173625,23.652639
11020.0,55600.0,95.0,95.000000,83.438600,0.666667,0.001709,14.621228,21.127098,2.413294


In [276]:
df_statecnty_stats.index = df_statecnty_stats.index.astype(int)

In [279]:
df_statecnty_stats

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud,pct_denied_apps,pct_rentals,pct_minority
msamd,,,,,,,,,
10180,52900.0,88.5,88.500000,20.506097,1.158423,0.001673,14.621228,28.400994,19.662451
10380,16600.0,119.0,110.200000,80.862229,1.157336,0.006639,14.621228,40.691458,98.921266
10420,66500.0,72.0,82.838710,48.747716,1.897519,0.001246,14.621228,19.649796,11.968323
10500,49400.0,81.5,82.250000,29.881711,2.063765,0.001665,14.621228,26.099862,22.874824
10580,78100.0,83.0,95.068182,63.521246,2.085460,0.001217,14.621228,24.502761,12.073518
10740,61900.0,64.0,82.882353,57.968476,3.394150,0.001339,14.621228,25.018652,52.440586
10780,52400.0,118.5,270.333333,421.120252,1.221641,0.005159,14.621228,31.147279,37.154751
10900,73300.0,73.0,80.106383,44.816648,2.576662,0.001093,14.621228,21.173625,23.652639
11020,55600.0,95.0,95.000000,83.438600,0.666667,0.001709,14.621228,21.127098,2.413294


In [278]:
#4) Plot one of these columns on a US county map as in Part 1.
map_fig = plt.figure(title='Visualizing Median Income by County in 2012', min_width=1300, min_height=800)
map_tt = Tooltip(fields=['name', 'color'], labels=['County Name'])
map_res = plt.geo(map_data=topo_load('map_data/USCountiesMap.json'), stroke_color='black',
                 colors={'default_color': 'Black'},
                  scales={'projection': AlbersUSA(),
                         'color': ColorScale(colors=['Red', 'Gray', 'DeepSkyBlue'])},
                  color=dict(zip(df_statecnty_stats.index, df_statecnty_stats['median_inc'])), tooltip=map_tt)
map_fig

A Jupyter Widget